In [47]:
import time
from collections import deque, namedtuple

#import gym
import gymnasium as gym
import numpy as np
import PIL.Image
import tensorflow as tf
import utils

from pyvirtualdisplay import Display
import torch
import torch.nn as nn

import public_tests
#from tensorflow.keras import Sequential
#from tensorflow.keras.layers import Dense, Input
#from tensorflow.keras.losses import MSE
#from tensorflow.keras.optimizers import Adam

In [14]:
Display(visible=0, size=(840, 480)).start();

torch.manual_seed(42)

In [15]:
MEMORY_SIZE = 100_000     # size of memory buffer
GAMMA = 0.995             # discount factor
ALPHA = 1e-3              # learning rate  
NUM_STEPS_FOR_UPDATE = 4  # perform a learning update every C time steps

In [16]:
print(MEMORY_SIZE)

100000


In [17]:
env = gym.make('LunarLander-v2')
env.reset()

state_size = env.observation_space.shape
num_actions = env.action_space.n

print('State Shape:', state_size)
print('Number of actions:', num_actions)

State Shape: (8,)
Number of actions: 4


In [18]:
# Select an action
action = 0

initial_state = env.reset()

# Run a single time step of the environment's dynamics with the given action.
next_state, reward, done, info, f = env.step(action)
#tuple = env.step(action)
print("The tuple")
print(tuple)

with np.printoptions(formatter={'float': '{:.3f}'.format}):
    print("Initial State:", initial_state)
    print("Action:", action)
    print("Next State:", next_state)
    print("Reward Received:", reward)
    print("Episode Terminated:", done)
    print("Info:", info)
    print("F :",f)

The tuple
<class 'tuple'>
Initial State: (array([-0.005, 1.419, -0.533, 0.370, 0.006, 0.121, 0.000, 0.000],
      dtype=float32), {})
Action: 0
Next State: [-0.011 1.427 -0.532 0.344 0.012 0.119 0.000 0.000]
Reward Received: 0.11857559850841426
Episode Terminated: False
Info: False
F : {}


In [30]:
# # UNQ_C1
# # GRADED CELL

# # Create the Q-Network
# print("State size :",type(state_size))
# q_network = Sequential([
#     ### START CODE HERE ### 
#     Input(shape=state_size),
#     Dense(units=64,activation='relu'),
#     Dense(units=64,activation='relu'),
#     Dense(units=num_actions,activation='linear'),
    
#     ### END CODE HERE ### 
#     ])

# # Create the target Q^-Network
# target_q_network = Sequential([
#     ### START CODE HERE ### 
#     Input(shape=state_size),
#     Dense(units=64,activation='relu'),
#     Dense(units=64,activation='relu'),
#     Dense(units=num_actions,activation='linear'),
#     ### END CODE HERE ###
#     ])

# ### START CODE HERE ### 
# optimizer = Adam(learning_rate=ALPHA)
# ### END CODE HERE ###

# print(q_network.summary(expand_nested=True))

# print(target_q_network.summary())

# this is how you'd presumably create a dense nn on torch using the same
# layout as tf
q_network = nn.Sequential(
    nn.Linear(state_size[0], 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 4),
    
)

target_q_network = nn.Sequential(
    nn.Linear(state_size[0], 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 4),
    
)

optimizer = torch.optim.Adam(q_network.parameters(), lr=ALPHA)
optimizer = torch.optim.Adam(target_q_network.parameters(), lr=ALPHA)


In [41]:
experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
np.random.seed(1)
states = np.float32(np.random.rand(64, 8))
actions = np.float32(np.floor(np.random.uniform(0, 1, (64, )) * 4))
rewards = np.float32(np.random.rand(64, ))
next_states = np.float32(np.random.rand(64, 8))
done_vals = np.float32((np.random.uniform(0, 1, size=(64,)) > 0.96) * 1)

print(actions)

[0. 0. 2. 1. 2. 0. 2. 3. 3. 2. 3. 3. 2. 0. 0. 0. 1. 1. 3. 0. 3. 3. 3. 2.
 2. 1. 0. 1. 1. 2. 1. 1. 0. 0. 3. 1. 3. 0. 1. 2. 3. 1. 0. 1. 2. 2. 3. 0.
 0. 2. 3. 0. 2. 2. 1. 3. 2. 0. 1. 3. 3. 2. 2. 2.]


In [108]:
# def compute_loss(experiences, gamma, q_network, target_q_network):
#     """ 
#     Calculates the loss.
    
#     Args:
#       experiences: (tuple) tuple of ["state", "action", "reward", "next_state", "done"] namedtuples
#       gamma: (float) The discount factor.
#       q_network: (tf.keras.Sequential) Keras model for predicting the q_values
#       target_q_network: (tf.keras.Sequential) Karas model for predicting the targets
          
#     Returns:
#       loss: (TensorFlow Tensor(shape=(0,), dtype=int32)) the Mean-Squared Error between
#             the y targets and the Q(s,a) values.
#     """
    
#     # Unpack the mini-batch of experience tuples
#     states, actions, rewards, next_states, done_vals = experiences
    
#     # Compute max Q^(s,a)
#     max_qsa = tf.reduce_max(target_q_network(next_states), axis=0)

#     print(type(target_q_network(next_states)))
    
#     print(target_q_network(next_states))
#     print("numpy shape ",target_q_network(next_states).shape)
#     print("numpy max, " ,np.max(target_q_network(next_states),axis=0))
#     print("numpy dtype , ",target_q_network(next_states).dtype)
#     print(max_qsa)
    
#     # Set y = R if episode terminates, otherwise set y = R + γ max Q^(s,a).
#     ### START CODE HERE ### 
#     y = (rewards + (1-done_vals)*(gamma*max_qsa))
#     ### END CODE HERE ###
    
#     # Get the q_values
#     q_values = q_network(states)
#     q_values = tf.gather_nd(q_values, tf.stack([tf.range(q_values.shape[0]),
#                                                 tf.cast(actions, tf.int32)], axis=1))
        
#     # Compute the loss
#     ### START CODE HERE ### 
#     loss = MSE(y, q_values)
#     ### END CODE HERE ### 
    
#     return loss

def compute_loss(experiences, gamma, q_network, target_q_network):
    
    # Unpack the mini-batch of experience tuples
    #print(experiences)
    states, actions, rewards, next_states, done_vals = experiences
    #input_tensor = torch.randn(20,20)
    #print(type(input_tensor))
    #print(input_tensor)

    print("END")
    n_statae = target_q_network(next_states)

    #print(n_statae)
    max_qsa , indicies  = torch.max(torch.from_numpy(n_statae), dim=1)
    
    print("Actions :",actions)

    print("max_qsa ", max_qsa)
    y = (rewards + (1-done_vals)*(gamma*max_qsa.numpy()))
    
    q_values = q_network(states)


    

    print("q_values ,",q_values)
    print("action shape ",type(actions))
    print(np.arange(actions.shape[0]))
    print("q_values 64, ",q_values[np.arange(actions.shape[0]),actions.astype(int)])
    q_values = q_values[np.arange(actions.shape[0]),actions.astype(int)]
    print("Y ,",y.shape)
    #print("q_values, ",q_values.size())
    mseloss = torch.nn.MSELoss()
    loss = mseloss(torch.from_numpy(y),torch.from_numpy(q_values))
    #mse_loss = torch.nn.functional.mse_loss(y,q_values)

    print(loss)
    return loss

public_tests.test_compute_loss(compute_loss)

END
Actions : [0. 0. 2. 1. 2. 0. 2. 3. 3. 2. 3. 3. 2. 0. 0. 0. 1. 1. 3. 0. 3. 3. 3. 2.
 2. 1. 0. 1. 1. 2. 1. 1. 0. 0. 3. 1. 3. 0. 1. 2. 3. 1. 0. 1. 2. 2. 3. 0.
 0. 2. 3. 0. 2. 2. 1. 3. 2. 0. 1. 3. 3. 2. 2. 2.]
max_qsa  tensor([0.9909, 0.7804, 0.9108, 0.8211, 0.9742, 0.8894, 0.8963, 0.8731, 0.9247,
        0.8850, 0.7176, 0.8370, 0.6384, 0.8805, 0.5914, 0.3029, 0.7417, 0.6784,
        0.9826, 0.8164, 0.7873, 0.9588, 0.7906, 0.6890, 0.9258, 0.7467, 0.9306,
        0.9787, 0.6606, 0.5380, 0.6963, 0.6873, 0.9796, 0.7647, 0.5348, 0.8010,
        0.9314, 0.9192, 0.9985, 0.9718, 0.8417, 0.9110, 0.7256, 0.8847, 0.9281,
        0.9769, 0.7104, 0.9904, 0.5269, 0.9332, 0.9595, 0.3127, 0.9537, 0.7581,
        0.8905, 0.9713, 0.9225, 0.8797, 0.4443, 0.6910, 0.8733, 0.9841, 0.2528,
        0.6400])
q_values , [[0.29469204 0.2696547  0.59054655 0.38053408]
 [0.70252305 0.49612394 0.6332093  0.35339436]
 [0.28704396 0.13320711 0.9370804  0.80483586]
 [0.9380285  0.18538052 0.9331558  0.9357532 ]
 [0.2